In [1]:
import pyaudio
import numpy as np
import whisper
import webbrowser

model = whisper.load_model("base")

In [2]:
def recognize_speech():
    record_duration = 5
    print("Говорите")
    p = pyaudio.PyAudio()
    stream = p.open(format=pyaudio.paInt16, channels=1, rate=44100, input=True, frames_per_buffer=44100)
    frames = []

    for _ in range(int(44100/1024 * record_duration)):
        data = stream.read(1024)
        frames.append(np.frombuffer(data, dtype=np.int16))

    stream.stop_stream()
    stream.close()
    p.terminate()

    audio = np.concatenate(frames)

    audio = whisper.pad_or_trim(audio)
    audio = audio.astype(np.float32)
    mel = whisper.log_mel_spectrogram(audio).to(model.device)

    
    _, probs = model.detect_language(mel)
    print(f"Detected language: {max(probs, key=probs.get)}")
    options = whisper.DecodingOptions()
    command = whisper.decode(model, mel, options)

    print(command.text)
    return command.text

In [3]:
command = recognize_speech().lower()

Говорите...
Detected language: en
I


In [4]:
if "Yandex" in command:
    search_query = command.replace("поиск в яндексе", "").strip()
    webbrowser.open(f"https://yandex.ru/search/?text={search_query}")
elif "Вконтакте" in command:
    webbrowser.open("https://vk.com/")
elif "Youtube" in command:
    search_query = command.replace("найти видео в youtube", "").strip()
    webbrowser.open(f"https://www.youtube.com/results?search_query={search_query}")
else:
    print("Команда не распознана.")

Команда не распознана.
